<a href="https://colab.research.google.com/github/guptaa98/Kaggle-Notebooks/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#It is a subset of a large 142.8 million Amazon review dataset that was made available by Stanford
#professor, Julian McAuley. This sentiment analysis dataset contains reviews from May 1996 to
#July 2014. The dataset reviews include ratings, text, helpful votes, product description, category
#information, price, brand, and image features. The complete dataset is available here:
#http://jmcauley.ucsd.edu/data/amazon/.
#worked on the smaller dataset of “SPORTS AND OUTDOORS”.

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from tqdm import tqdm

In [ ]:
project_data = pd.read_json('/content/drive/My Drive/nlpassignment/Sports_and_Outdoors_5.json', lines=True )

In [ ]:
project_data.head() 

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


In [ ]:
print("Number of data points in train data", project_data.shape)
print('-'*50)
print("The attributes of data :", project_data.columns.values)

Number of data points in train data (296337, 9)
--------------------------------------------------
The attributes of data : ['reviewerID' 'asin' 'reviewerName' 'helpful' 'reviewText' 'overall'
 'summary' 'unixReviewTime' 'reviewTime']


In [ ]:
approved_project = project_data['overall'].values   #contains the labels
project_data.drop(['overall'], axis=1, inplace=True)
project_data.head(1)

,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,Woks very good,1390694400,"01 26, 2014"


In [ ]:
project_data.drop(['reviewTime'], axis=1, inplace=True)
project_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,summary,unixReviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,Woks very good,1390694400
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,Works as well as the factory tool,1328140800
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,"It's a punch, that's all.",1330387200
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,It's a punch with a Glock logo.,1328400000
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,"Ok,tool does what a regular punch does.",1366675200


In [ ]:
# Data splitting

from sklearn.model_selection import train_test_split

# Splitting in train and test
X_train, X_test, y_train, y_test = train_test_split(project_data, approved_project, test_size=0.33, stratify=approved_project)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(198545, 7)
(97792, 7)
(198545,)
(97792,)


In [ ]:
#adding values of reviewText in tokenization for finding top 10 most occuring words
tokenization = ""
for dataset in tqdm(X_train['reviewText'].values):
        tokenization += dataset

100%|██████████| 198545/198545 [00:00<00:00, 597169.82it/s]


In [ ]:
#adding values of reviewText in tokenization for finding top 10 most occuring words
tokenization = ""
for dataset in tqdm(X_test['reviewText'].values):
        tokenization += dataset

100%|██████████| 97792/97792 [00:00<00:00, 584191.16it/s]


In [ ]:
#finding top 10 words
from collections import Counter
split_it = tokenization.split()
Counter = Counter(split_it)
most_occur = Counter.most_common(10)
print(most_occur)

[('the', 375835), ('a', 239857), ('and', 236669), ('to', 221002), ('I', 196542), ('it', 149952), ('is', 141630), ('of', 122723), ('for', 116545), ('in', 89638)]


In [ ]:
#stemming on training set
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()

preprocessed_text_train = []
# tqdm is for printing the status bar
for sentence in tqdm(X_train['reviewText'].values):
    sent = word_tokenize(sentence)      #tokenization
    
    stem_word = ""
    for word in sent:
        stem_word =  stem_word + " " + stemmer.stem(word)
    
    preprocessed_text_train.append(stem_word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


100%|██████████| 198545/198545 [07:23<00:00, 447.90it/s]


In [ ]:
#stemming on test set
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()

preprocessed_text_test = []
# tqdm is for printing the status bar
for sentence in tqdm(X_test['reviewText'].values):
    sent = word_tokenize(sentence)        #tokenization
    
    stem_word = ""
    for word in sent:
        stem_word =  stem_word + " " + stemmer.stem(word)
    
    preprocessed_text_test.append(stem_word)

  0%|          | 31/97792 [00:00<05:48, 280.12it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 97792/97792 [03:38<00:00, 448.34it/s]


In [ ]:
#lemmatization on training set
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

lemmatized_text_train = []
for i in range (0,len(preprocessed_text_train)):
    
    input_str = word_tokenize(preprocessed_text_train[i])
    
    lemmatize_word = ""
    for word in input_str:
        lemmatize_word = lemmatize_word + " " + lemmatizer.lemmatize(word)
    
    lemmatized_text_train.append(lemmatize_word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#lemmatization on test set
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

lemmatized_text_test = []
for i in range (0,len(preprocessed_text_test)):
    
    input_str = word_tokenize(preprocessed_text_test[i])
    
    lemmatize_word = ""
    for word in input_str:
        lemmatize_word = lemmatize_word + " " + lemmatizer.lemmatize(word)
    
    lemmatized_text_test.append(lemmatize_word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
#cleaning on train set
final_preprocessed_text_train = []

for i in range (0,len(lemmatized_text_train)):
    sent = decontracted(lemmatized_text_train[i])
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent = re.sub("\S*\d\S*", "", sent).strip()
    sent = re.sub(r"@", '', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords)  
    final_preprocessed_text_train.append(sent.lower().strip())

In [ ]:
#cleaning on test set
final_preprocessed_text_test = []

for i in range (0,len(lemmatized_text_test)):
    sent = decontracted(lemmatized_text_train[i])
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent = re.sub("\S*\d\S*", "", sent).strip()
    sent = re.sub(r"@", '', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords)  
    final_preprocessed_text_test.append(sent.lower().strip())

In [ ]:
for i in range (0,21):
    print(final_preprocessed_text_train[i])
    print("="*50)

great pump spec discus post lezyn pressur drive pump today not smaller tire though hose still rock valv tho le
not smith driver fit screw well torx ad extra i rate thi becaus bit size fit could better i give thi set go thank
doe job small handi stay bugout bag
great much better mre i sinc much lighter mre it emerg suppli food though must rememb enough clean filter water
thi work exactli would expect look exactli pictur i also fobu handcuff singl mag paddl setup find i prefer use not becaus wa anyth wrong thi hold two full size glock mag secur tight enough wo not fall upsid not tight give ani troupl draw
i bought return thi item onli becaus wa small my wa not list gun would fit i measur gun felt sure would fit base dimens mention product descript gun fit i would gladli kept it wa cheap seem veri nice price also i want mention amazon made return veri easi quick
came veri sharp love length fact blade secur full length handl grip veri well even wet would give star wa disappoint serrat side

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(final_preprocessed_text_train)
reviewText_bow_train = vectorizer.transform(final_preprocessed_text_train)
reviewText_bow_test = vectorizer.transform(final_preprocessed_text_test)
print("Shape of matrix after one hot encoding ",reviewText_bow_train.shape)
print("Shape of matrix after one hot encoding ",reviewText_bow_test.shape)

Shape of matrix after one hot encoding  (198545, 13167)
Shape of matrix after one hot encoding  (97792, 13167)


In [ ]:
X_train.drop(['reviewText'],axis=1,inplace=True)
X_train['reviewText']=reviewText_bow_train
X_train.head()

,reviewerID,asin,reviewerName,helpful,summary,unixReviewTime,reviewText
245544,A3O1OLQ5PSS42D,B005QYHG7K,Tony,"[0, 0]",High volume!,1334707200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
164470,A12DIV3DX2I3EA,B002PQKGSI,Rikki Rocktrap,"[0, 0]",Not Smith Drivers,1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
60413,AUWRF830PO35J,B000NMLXBA,"Jerome Bruss ""djbruss""","[0, 0]","Does the job! Small, and, handy. ...",1404691200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
42479,A24U05FKTVCWVU,B000G2TOMM,rhoffa,"[1, 1]",Truly Almost perfect!,1323302400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
5506,A3VPP95ZKOOUJK,B0000C53LF,Chain of Command,"[0, 0]",works as advertised,1386547200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."


In [ ]:
X_test.drop(['reviewText'],axis=1,inplace=True)
X_test['reviewText']=reviewText_bow_test
X_test.head()

,reviewerID,asin,reviewerName,helpful,summary,unixReviewTime,reviewText
221133,A3DOPYDOS49I3T,B004O844DS,Confucious,"[62, 66]",Excellent cargo rack for the price,1319068800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
263359,A1AFY8WLHKC20T,B007WHCFC0,Morthacon,"[0, 0]",Loving this!,1385596800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
274556,AD0WGOZ5CGP02,B0093IS0GQ,Jones,"[11, 11]",Klean Kanteen Wide Mouth Bottle with Stainless...,1363478400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
271351,A35LT14WG5BX0T,B008NQF6RA,Lola Bunny,"[0, 0]",Surprisingly effective!,1375401600,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."
45577,A1F4JPGWHYHU71,B000H6GZI8,Daniel E. Lechliter,"[0, 0]",Houge Rubber Grip,1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1..."


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
reviewerID_bow_train = vectorizer.fit(X_train['reviewerID'])
reviewerID_bow_train = vectorizer.transform(X_train['reviewerID'])
reviewerID_bow_test = vectorizer.transform(X_test['reviewerID'])
print("Shape of matrix after one hot encoding ",reviewerID_bow_test.shape)
print("Shape of matrix after one hot encoding ",reviewerID_bow_train.shape)

Shape of matrix after one hot encoding  (97792, 3420)
Shape of matrix after one hot encoding  (198545, 3420)


In [ ]:
X_train.drop(['reviewerID'],axis=1,inplace=True)
X_train['reviewerID']=reviewerID_bow_train
X_train.head()

,asin,reviewerName,helpful,summary,unixReviewTime,reviewText,reviewerID
245544,B005QYHG7K,Tony,"[0, 0]",High volume!,1334707200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1..."
164470,B002PQKGSI,Rikki Rocktrap,"[0, 0]",Not Smith Drivers,1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1..."
60413,B000NMLXBA,"Jerome Bruss ""djbruss""","[0, 0]","Does the job! Small, and, handy. ...",1404691200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1..."
42479,B000G2TOMM,rhoffa,"[1, 1]",Truly Almost perfect!,1323302400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1..."
5506,B0000C53LF,Chain of Command,"[0, 0]",works as advertised,1386547200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1..."


In [ ]:
X_test.drop(['reviewerID'],axis=1,inplace=True)
X_test['reviewerID']=reviewerID_bow_test
X_test.head()

,asin,reviewerName,helpful,summary,unixReviewTime,reviewText,reviewerID
221133,B004O844DS,Confucious,"[62, 66]",Excellent cargo rack for the price,1319068800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\..."
263359,B007WHCFC0,Morthacon,"[0, 0]",Loving this!,1385596800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\..."
274556,B0093IS0GQ,Jones,"[11, 11]",Klean Kanteen Wide Mouth Bottle with Stainless...,1363478400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\..."
271351,B008NQF6RA,Lola Bunny,"[0, 0]",Surprisingly effective!,1375401600,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\..."
45577,B000H6GZI8,Daniel E. Lechliter,"[0, 0]",Houge Rubber Grip,1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['summary'])
summary_bow_train = vectorizer.transform(X_train['summary'])
summary_bow_test = vectorizer.transform(X_test['summary'])
print("Shape of matrix after one hot encoding ",summary_bow_train.shape)
print("Shape of matrix after one hot encoding ",summary_bow_test.shape)


Shape of matrix after one hot encoding  (198545, 4409)
Shape of matrix after one hot encoding  (97792, 4409)


In [ ]:
X_train.drop(['summary'],axis=1,inplace=True)
X_train['summary']=summary_bow_train
X_train.head()

,asin,reviewerName,helpful,unixReviewTime,reviewText,reviewerID,summary
245544,B005QYHG7K,Tony,"[0, 0]",1334707200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
164470,B002PQKGSI,Rikki Rocktrap,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
60413,B000NMLXBA,"Jerome Bruss ""djbruss""","[0, 0]",1404691200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
42479,B000G2TOMM,rhoffa,"[1, 1]",1323302400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
5506,B0000C53LF,Chain of Command,"[0, 0]",1386547200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."


In [ ]:
X_test.drop(['summary'],axis=1,inplace=True)
X_test['summary']=summary_bow_train
X_test.head()

,asin,reviewerName,helpful,unixReviewTime,reviewText,reviewerID,summary
221133,B004O844DS,Confucious,"[62, 66]",1319068800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
263359,B007WHCFC0,Morthacon,"[0, 0]",1385596800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
274556,B0093IS0GQ,Jones,"[11, 11]",1363478400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
271351,B008NQF6RA,Lola Bunny,"[0, 0]",1375401600,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."
45577,B000H6GZI8,Daniel E. Lechliter,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1..."


In [ ]:
X_train["reviewerName"].fillna("No name", inplace = True) 

In [ ]:
X_test["reviewerName"].fillna("No name", inplace = True) 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['reviewerName'])
reviewerName_bow_train = vectorizer.transform(X_train['reviewerName'])
reviewerName_bow_test = vectorizer.transform(X_test['reviewerName'])
print("Shape of matrix after one hot encoding ",reviewerName_bow_test.shape)
print("Shape of matrix after one hot encoding ",reviewerName_bow_train.shape)

Shape of matrix after one hot encoding  (97792, 5782)
Shape of matrix after one hot encoding  (198545, 5782)


In [ ]:
X_train.drop(['reviewerName'],axis=1,inplace=True)
X_train['reviewerName']=reviewerName_bow_train
X_train.head()

,asin,helpful,unixReviewTime,reviewText,reviewerID,summary,reviewerName
245544,B005QYHG7K,"[0, 0]",1334707200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\..."
164470,B002PQKGSI,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\..."
60413,B000NMLXBA,"[0, 0]",1404691200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\..."
42479,B000G2TOMM,"[1, 1]",1323302400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\..."
5506,B0000C53LF,"[0, 0]",1386547200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\..."


In [ ]:
X_test.drop(['reviewerName'],axis=1,inplace=True)
X_test['reviewerName']=reviewerName_bow_test
X_test.head()

,asin,helpful,unixReviewTime,reviewText,reviewerID,summary,reviewerName
221133,B004O844DS,"[62, 66]",1319068800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\..."
263359,B007WHCFC0,"[0, 0]",1385596800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\..."
274556,B0093IS0GQ,"[11, 11]",1363478400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\..."
271351,B008NQF6RA,"[0, 0]",1375401600,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\..."
45577,B000H6GZI8,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['asin'])
asin_bow_train = vectorizer.transform(X_train['asin'])
asin_bow_test = vectorizer.transform(X_test['asin'])
print("Shape of matrix after one hot encoding ",asin_bow_test.shape)
print("Shape of matrix after one hot encoding ",asin_bow_train.shape)

Shape of matrix after one hot encoding  (97792, 5456)
Shape of matrix after one hot encoding  (198545, 5456)


In [ ]:
X_train.drop(['asin'],axis=1,inplace=True)
X_train['asin']=asin_bow_train
X_train.head()

,helpful,unixReviewTime,reviewText,reviewerID,summary,reviewerName,asin
245544,"[0, 0]",1334707200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\..."
164470,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\..."
60413,"[0, 0]",1404691200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\..."
42479,"[1, 1]",1323302400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\..."
5506,"[0, 0]",1386547200,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\..."


In [ ]:
X_test.drop(['asin'],axis=1,inplace=True)
X_test['asin']=asin_bow_test
X_test.head()

,helpful,unixReviewTime,reviewText,reviewerID,summary,reviewerName,asin
221133,"[62, 66]",1319068800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1..."
263359,"[0, 0]",1385596800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1..."
274556,"[11, 11]",1363478400,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1..."
271351,"[0, 0]",1375401600,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1..."
45577,"[0, 0]",1360108800,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1..."


In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['unixReviewTime'].values.reshape(-1,1))

X_train_price_norm = normalizer.transform(X_train['unixReviewTime'].values.reshape(-1,1))
X_test_price_norm = normalizer.transform(X_test['unixReviewTime'].values.reshape(-1,1))

In [ ]:
X_test.drop(['unixReviewTime'],axis=1,inplace=True)
X_test['unixReviewTime']=X_test_price_norm
X_test.head()

,helpful,reviewText,reviewerID,summary,reviewerName,asin,unixReviewTime
221133,"[62, 66]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
263359,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
274556,"[11, 11]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
271351,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
45577,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0


In [ ]:
X_train.drop(['unixReviewTime'],axis=1,inplace=True)
X_train['unixReviewTime']=X_train_price_norm
X_train.head()

,helpful,reviewText,reviewerID,summary,reviewerName,asin,unixReviewTime
245544,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
164470,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
60413,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
42479,"[1, 1]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
5506,"[0, 0]","(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0


In [ ]:
X_train.drop(['helpful'],axis=1,inplace=True)

In [ ]:
X_test.drop(['helpful'],axis=1,inplace=True)

In [ ]:
X_train.head()

,reviewText,reviewerID,summary,reviewerName,asin,unixReviewTime
245544,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
164470,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
60413,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
42479,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0
5506,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(2, 3304)\t1\n (6, 2539)\t1\n (8, 2031)\t1...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 5277)\t1\n (2, 720)\t1\n (2, 1475)\t1\...","(1, 3021)\t1\n (2, 1154)\t1\n (3, 828)\t1\...",1.0


In [ ]:
X_test.head()

,reviewText,reviewerID,summary,reviewerName,asin,unixReviewTime
221133,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
263359,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
274556,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
271351,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0
45577,"(0, 3133)\t1\n (0, 3340)\t1\n (0, 4839)\t1...","(0, 2140)\t1\n (5, 1787)\t1\n (8, 833)\t1\...","(0, 1827)\t1\n (0, 4191)\t1\n (1, 1209)\t1...","(0, 1120)\t1\n (2, 2764)\t1\n (3, 751)\t1\...","(0, 4086)\t1\n (1, 4840)\t1\n (2, 5039)\t1...",1.0


In [ ]:
from scipy.sparse import hstack

# Train data stack
X_tr = hstack((reviewText_bow_train,reviewerID_bow_train,summary_bow_train,reviewerName_bow_train,asin_bow_train,X_train_price_norm)).tocsr()


# Test Data Stack
X_te = hstack((reviewText_bow_test,reviewerID_bow_test,summary_bow_test,reviewerName_bow_test,asin_bow_test,X_test_price_norm)).tocsr()

print("Final Data matrix")
print(X_tr.shape, y_train.shape)

print(X_te.shape, y_test.shape)
print("="*100)

Final Data matrix
(198545, 32235) (198545,)
(97792, 32235) (97792,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_tr,y_train)
pred = clf.predict(X_te)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)

0.5616819371727748

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[  467,   212,   408,   571,  1327],
       [  278,   225,   609,   762,  1493],
       [  376,   295,  1518,  2216,  3539],
       [  679,   402,  1693,  5991, 12622],
       [ 1835,  1011,  2422, 10114, 46727]])

In [ ]:
#precision, recall ad F1 Score 
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred, average='weighted')

(0.5443896723436595, 0.5616819371727748, 0.5522321657087909, None)